In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import urllib

In [2]:
census = pd.read_csv('http://www2.census.gov/geo/docs/reference/codes/files/national_county.txt',sep=',',header=None, dtype=str)
census.columns = ['state_abbr', 'state_fips', 'county_fips', 'county_name', 'fips_class_code']
#print(census.shape)
census.head(20)

,state_abbr,state_fips,county_fips,county_name,fips_class_code
0,AL,01,001,Autauga County,H1
1,AL,01,003,Baldwin County,H1
2,AL,01,005,Barbour County,H1
3,AL,01,007,Bibb County,H1
4,AL,01,009,Blount County,H1
5,AL,01,011,Bullock County,H1
6,AL,01,013,Butler County,H1
7,AL,01,015,Calhoun County,H1
8,AL,01,017,Chambers County,H1
9,AL,01,019,Cherokee County,H1


In [3]:
file_to_load = pd.read_csv('countypres_2000-2016.csv')

countpres = pd.DataFrame(file_to_load)
countpres

,year,state,state_po,county,FIPS,office,candidate,party,candidatevotes,totalvotes,version
0,2000,Alabama,AL,Autauga,1001.0,President,Al Gore,democrat,4942.0,17208,20191203
1,2000,Alabama,AL,Autauga,1001.0,President,George W. Bush,republican,11993.0,17208,20191203
2,2000,Alabama,AL,Autauga,1001.0,President,Ralph Nader,green,160.0,17208,20191203
3,2000,Alabama,AL,Autauga,1001.0,President,Other,NaN,113.0,17208,20191203
4,2000,Alabama,AL,Baldwin,1003.0,President,Al Gore,democrat,13997.0,56480,20191203
...,...,...,...,...,...,...,...,...,...,...,...
50511,2016,Alaska,AK,District 40,2040.0,President,Donald Trump,republican,1377.0,4610,20191203
50512,2016,Alaska,AK,District 40,2040.0,President,Other,NaN,895.0,4610,20191203
50513,2016,Alaska,AK,District 99,2099.0,President,Hillary Clinton,democrat,274.0,5056,20191203
50514,2016,Alaska,AK,District 99,2099.0,President,Donald Trump,republican,40.0,5056,20191203


In [4]:
countpres.FIPS =countpres.FIPS.fillna(0)
countpres['FIPS']= countpres['FIPS'].astype(int)
countpres = countpres[countpres['FIPS'] != '0']
#countpres = countpres.dropna(subset=['FIPS'], inplace=True)
countpres.county = countpres.county + ' County'
countpres.dropna(subset =['state_po'], inplace=True)
countpres.rename(columns={'state_po': "state_abbr"}, inplace=True)
countpres.drop(columns=['office', 'version'])

,year,state,state_abbr,county,FIPS,candidate,party,candidatevotes,totalvotes
0,2000,Alabama,AL,Autauga County,1001,Al Gore,democrat,4942.0,17208
1,2000,Alabama,AL,Autauga County,1001,George W. Bush,republican,11993.0,17208
2,2000,Alabama,AL,Autauga County,1001,Ralph Nader,green,160.0,17208
3,2000,Alabama,AL,Autauga County,1001,Other,NaN,113.0,17208
4,2000,Alabama,AL,Baldwin County,1003,Al Gore,democrat,13997.0,56480
...,...,...,...,...,...,...,...,...,...
50511,2016,Alaska,AK,District 40 County,2040,Donald Trump,republican,1377.0,4610
50512,2016,Alaska,AK,District 40 County,2040,Other,NaN,895.0,4610
50513,2016,Alaska,AK,District 99 County,2099,Hillary Clinton,democrat,274.0,5056
50514,2016,Alaska,AK,District 99 County,2099,Donald Trump,republican,40.0,5056


In [10]:
countpres['combined'] = countpres['state_abbr'] + countpres['county'].apply(lambda x: x.replace(' ','').lower())
countpres[(countpres['state_abbr'] == 'DC')]
#countpres = countpres.drop(columns='version', axis=1)
countpres

,year,state,state_abbr,county,FIPS,office,candidate,party,candidatevotes,totalvotes,combined
0,2000,Alabama,AL,Autauga County,1001,President,Al Gore,democrat,4942.0,17208,ALautaugacounty
1,2000,Alabama,AL,Autauga County,1001,President,George W. Bush,republican,11993.0,17208,ALautaugacounty
2,2000,Alabama,AL,Autauga County,1001,President,Ralph Nader,green,160.0,17208,ALautaugacounty
3,2000,Alabama,AL,Autauga County,1001,President,Other,NaN,113.0,17208,ALautaugacounty
4,2000,Alabama,AL,Baldwin County,1003,President,Al Gore,democrat,13997.0,56480,ALbaldwincounty
...,...,...,...,...,...,...,...,...,...,...,...
50511,2016,Alaska,AK,District 40 County,2040,President,Donald Trump,republican,1377.0,4610,AKdistrict40county
50512,2016,Alaska,AK,District 40 County,2040,President,Other,NaN,895.0,4610,AKdistrict40county
50513,2016,Alaska,AK,District 99 County,2099,President,Hillary Clinton,democrat,274.0,5056,AKdistrict99county
50514,2016,Alaska,AK,District 99 County,2099,President,Donald Trump,republican,40.0,5056,AKdistrict99county


In [128]:
census[(census['state_abbr'] == 'AL') & (census['county_name'] == 'Autauga County')]
cenus[]

,state_abbr,state_fips,county_fips,county_name,fips_class_code
0,AL,01,001,Autauga County,H1


In [12]:
census['combined'] = census['state_abbr'] + census['county_name'].apply(lambda x: x.replace(' ','').lower())
census

,state_abbr,state_fips,county_fips,county_name,fips_class_code,combined
0,AL,01,001,Autauga County,H1,ALautaugacounty
1,AL,01,003,Baldwin County,H1,ALbaldwincounty
2,AL,01,005,Barbour County,H1,ALbarbourcounty
3,AL,01,007,Bibb County,H1,ALbibbcounty
4,AL,01,009,Blount County,H1,ALblountcounty
...,...,...,...,...,...,...
3230,PR,72,153,Yauco Municipio,H1,PRyaucomunicipio
3231,UM,74,300,Midway Islands,H4,UMmidwayislands
3232,VI,78,010,St. Croix Island,H4,VIst.croixisland
3233,VI,78,020,St. John Island,H4,VIst.johnisland


In [16]:
left = countpres.set_index('combined')
right = census.set_index('combined')
combined = left.join(right, lsuffix='', rsuffix='_r')
combined = combined.reset_index()
combined[(combined['state_abbr'] == 'AL')]

,combined,year,state,state_abbr,county,FIPS,office,candidate,party,candidatevotes,totalvotes,state_abbr_r,state_fips,county_fips,county_name,fips_class_code
647,ALautaugacounty,2000,Alabama,AL,Autauga County,1001,President,Al Gore,democrat,4942.0,17208,AL,01,001,Autauga County,H1
648,ALautaugacounty,2000,Alabama,AL,Autauga County,1001,President,George W. Bush,republican,11993.0,17208,AL,01,001,Autauga County,H1
649,ALautaugacounty,2000,Alabama,AL,Autauga County,1001,President,Ralph Nader,green,160.0,17208,AL,01,001,Autauga County,H1
650,ALautaugacounty,2000,Alabama,AL,Autauga County,1001,President,Other,NaN,113.0,17208,AL,01,001,Autauga County,H1
651,ALautaugacounty,2004,Alabama,AL,Autauga County,1001,President,John Kerry,democrat,4758.0,20081,AL,01,001,Autauga County,H1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1714,ALwinstoncounty,2012,Alabama,AL,Winston County,1133,President,Mitt Romney,republican,8312.0,9714,AL,01,133,Winston County,H1
1715,ALwinstoncounty,2012,Alabama,AL,Winston County,1133,President,Other,NaN,116.0,9714,AL,01,133,Winston County,H1
1716,ALwinstoncounty,2016,Alabama,AL,Winston County,1133,President,Hillary Clinton,democrat,872.0,10313,AL,01,133,Winston County,H1
1717,ALwinstoncounty,2016,Alabama,AL,Winston County,1133,President,Donald Trump,republican,9228.0,10313,AL,01,133,Winston County,H1
